# Friends Classification Text

This is classification of dialogue in FRIENDS TV Series.

<!-- https://stackoverflow.com/questions/61000500/tensorflow-keras-bert-multiclass-text-classification-accuracy -->

In [1]:
import pandas as pd
import re
import numpy as np
import time

import os
import pprint
import tensorflow as tf

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# help(drive.mount)

In [3]:
PATH_DATA = '../../data/'

## Checking Machine

In [4]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

TPU address is grpc://10.62.78.122:8470


## Importing Data

In [5]:
%cd gdrive/My Drive/Repository/the-one-with-friends/notebooks/bert_model

/content/gdrive/My Drive/Repository/the-one-with-friends/notebooks/bert_model


In [6]:
os.listdir()

['.ipynb_checkpoints',
 '20200803_Data_Preparation.ipynb',
 '20200803_training_bert.ipynb',
 'bert_learn.ipynb',
 '20200803_training_bert_TPU.ipynb']

In [7]:
df_dlg_seen = pd.read_csv('../../data/friends_seen_season.csv')
df_dlg_unseen = pd.read_csv('../../data/friends_unseen_season.csv')

In [8]:
data_train = df_dlg_seen[df_dlg_seen.is_train == True][['text','label']]
data_test = df_dlg_seen[df_dlg_seen.is_train == False][['text','label']]
data_unseen = df_dlg_unseen[['text','label']]

In [9]:
data_train.shape

(35123, 2)

In [10]:
data_test.shape

(11708, 2)

In [11]:
data_unseen.shape

(11207, 2)

## Tokenizer

In [17]:
# !pip install transformers

In [18]:
# from transformers import BertTokenizer

# bert_token = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# MAX_LEN = 25
# NUMB_CLASS = 7

# # Tokenizer use bert
# def text_to_feature(x,tokenizer = bert_token):
#     text_token = tokenizer.encode_plus(
#                     x,                      
#                     add_special_tokens = True, # add [CLS], [SEP]
#                     max_length = MAX_LEN, # max length of the text that can go to BERT
#                     pad_to_max_length = True, # add [PAD] tokens
#                     return_attention_mask = True,
#                     truncation=True,# add attention mask to not focus on pad tokens
#                   )
#     return text_token

# # map to the expected input to TFBertForSequenceClassification, see here 
# def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
#     return {
#       "input_ids": input_ids,
#       "token_type_ids": token_type_ids,
#       "attention_mask": attention_masks,
#     }, label


# def label_encode(i, numb_cls):
#     X = np.zeros(numb_cls)
#     X[i] = 1
#     return list(X)

# def encode_dataset(ds, limit=-1):
#     # prepare list, so that we can build up final TensorFlow dataset from slices.
#     input_ids_list = []
#     token_type_ids_list = []
#     attention_mask_list = []
#     label_list = []
#     if (limit > 0):
#         ds = ds.head(limit)
    
#     for text, label in ds.values:
#         bert_input = text_to_feature(text)
#         input_ids_list.append(bert_input['input_ids'])
#         token_type_ids_list.append(bert_input['token_type_ids'])
#         attention_mask_list.append(bert_input['attention_mask'])
        
#     return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [19]:
from transformers import BertTokenizer

bert_token = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenizer use bert
def text_to_feature(x,max_seq,tokenizer = bert_token):
    text_token = tokenizer.encode_plus(
                    x,                      
                    add_special_tokens = True, # add [CLS], [SEP]
                    max_length = max_seq, # max length of the text that can go to BERT
                    pad_to_max_length = True, # add [PAD] tokens
                    return_attention_mask = True,
                    truncation=True,# add attention mask to not focus on pad tokens

                    return_tensors='tf',
                  )
    return text_token

def label_encode(i, numb_cls):
    X = np.zeros(numb_cls)
    X[i] = 1
    return list(X)

def encode_dataset(ds, max_seq_len, limit=-1):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.head(limit)
    
    for text, label in ds.values:
        bert_input = text_to_feature(text,max_seq = max_seq_len)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(label_encode(label, NUMB_CLASS))

    input_ids = tf.convert_to_tensor(input_ids_list)
    attention_masks = tf.convert_to_tensor(attention_mask_list)
    token_type_ids = tf.convert_to_tensor(token_type_ids_list)

    ids = tf.reshape(input_ids, (-1, max_seq_len))
    print("Input ids shape: ", ids.shape)
    masks = tf.reshape(attention_masks, (-1, max_seq_len))
    print("Input Masks shape: ", masks.shape)
    token_types = tf.reshape(token_type_ids, (-1, max_seq_len))
    print("Token type ids shape: ", token_types.shape)

    ids=ids.numpy()
    masks = masks.numpy()
    token_types = token_types.numpy()
    y = np.array(label_list)
        
    return [ids, masks, token_types, y]


In [20]:
# ALL_TEST = encode_dataset(data_test, MAX_LEN)
# ALL_TEST[:3],ALL_TEST[3].shape

## Training Model

In [26]:
from transformers import TFBertForSequenceClassification
from keras import Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras import regularizers

In [27]:
# def create_model(L_RATE, NUMB_CLASS_TARGET):
#   model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUMB_CLASS_TARGET)
#   optimizer = tf.keras.optimizers.Adam(learning_rate=L_RATE, epsilon=1e-08)

#   loss = tf.keras.losses.CategoricalCrossentropy()
#   metric = tf.keras.metrics.CategoricalCrossentropy('accuracy')

#   model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

#   return model


def create_model(L_RATE,max_seq_len,NUMB_CLASS_TARGET):
  base_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', trainable=True, num_labels=NUMB_CLASS_TARGET)

  input_ids_layer = Input(shape=(max_seq_len, ), dtype=np.int32)
  input_mask_layer = Input(shape=(max_seq_len, ), dtype=np.int32)
  input_token_type_layer = Input(shape=(max_seq_len,), dtype=np.int32)

  bert_layer = base_model([input_ids_layer, input_mask_layer, input_token_type_layer])[0]
  flat_layer = Flatten()(bert_layer)
  dropout_1= Dropout(0.5)(flat_layer)
  dense_1 = Dense(3*NUMB_CLASS_TARGET, activation='relu', 
                  bias_regularizer=regularizers.l2(0.05),
                  activity_regularizer=regularizers.l2(0.05))(dropout_1)

  dropout_2= Dropout(0.5)(dense_1)
  dense_2 = Dense(3*NUMB_CLASS_TARGET, activation='relu',
                  bias_regularizer=regularizers.l2(0.03),
                  activity_regularizer=regularizers.l2(0.03))(dropout_2)

  dense_output = Dense(NUMB_CLASS_TARGET, activation='softmax',
                  bias_regularizer=regularizers.l2(0.02))(dense_2)

  model_ = Model(inputs=[input_ids_layer, input_mask_layer, input_token_type_layer], outputs=dense_output)

  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=L_RATE,
    decay_steps=100000,
    decay_rate=0.9)
  
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, epsilon=1e-08)
  model_.compile(optimizer=optimizer,
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])
  
  #loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  #metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  #model_.compile(optimizer=optimizer, loss=loss, metrics=[metric])
  return model_


In [28]:
# mm = create_model(LR, MAX_LEN , NUMB_CLASS)

In [29]:
print(tf.__version__)

2.3.0


In [30]:
# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
LR = 1e-5
EPOCHS = 230
BATCH_SIZE = 250
MAX_LEN = 15
NUMB_CLASS = 7

use_tpu = 'COLAB_TPU_ADDR' in os.environ
if use_tpu:
    print('USING TPU')
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        bert_model = create_model(LR, MAX_LEN , NUMB_CLASS)
else:
    print('NOT USING TPU')
    bert_model = create_model(LR, MAX_LEN, NUMB_CLASS)

USING TPU


INFO:tensorflow:Initializing the TPU system: grpc://10.62.78.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.62.78.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
bert_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica ((None, 7),)         109487623   input_4[0][0]                    
                                                                 input_5[0][0]         

In [32]:
# XX = encode_dataset(data_test)
# YY = data_test['label'].tolist()
# ALL_TEST = convert_inputs_to_tf_dataset(XX,YY, MAX_LEN)
# ALL_TEST[:3],ALL_TEST[3].shape

In [33]:
L1 = time.time()
all_train = encode_dataset(data_train, MAX_LEN)
X_train,y_train = all_train[:3],all_train[3]
finish_time = str(round((time.time()-L1)/60,3))
print('done in '+finish_time)

L1 = time.time()
all_val = encode_dataset(data_test, MAX_LEN)
X_val,y_val = all_val[:3],all_val[3]
finish_time = str(round((time.time()-L1)/60,3))
print('done in '+finish_time)

L1 = time.time()
all_unseen = encode_dataset(data_unseen, MAX_LEN)
X_unseen,y_unseen = all_unseen[:3],all_unseen[3]
finish_time = str(round((time.time()-L1)/60,3))
print('done in '+finish_time)


Input ids shape:  (35123, 15)
Input Masks shape:  (35123, 15)
Token type ids shape:  (35123, 15)
done in 0.501
Input ids shape:  (11708, 15)
Input Masks shape:  (11708, 15)
Token type ids shape:  (11708, 15)
done in 0.182
Input ids shape:  (11207, 15)
Input Masks shape:  (11207, 15)
Token type ids shape:  (11207, 15)
done in 0.166


In [34]:
# optimizer = tf.keras.optimizers.Adam(learning_rate = 2e-5, epsilon=1e-08)
# bert_model.compile(optimizer=optimizer,
#               loss=['categorical_crossentropy'],
#               metrics=['accuracy'])



In [36]:
# bert_history = bert_model.fit(input_train, epochs=EPOCHS, validation_data=input_test)

bert_history = bert_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size = BATCH_SIZE, verbose=1)


Epoch 1/230
  2/141 [..............................] - ETA: 13s - loss: 1.3520 - accuracy: 0.5500WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0096s vs `on_train_batch_end` time: 0.0881s). Check your callbacks.


141/141 [==============================] - ETA: 0s - loss: 1.3337 - accuracy: 0.5595WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0044s vs `on_test_batch_end` time: 0.0310s). Check your callbacks.


141/141 [==============================] - 16s 113ms/step - loss: 1.3337 - accuracy: 0.5595 - val_loss: 2.2530 - val_accuracy: 0.2162
Epoch 2/230
141/141 [==============================] - 16s 116ms/step - loss: 1.3306 - accuracy: 0.5596 - val_loss: 2.2951 - val_accuracy: 0.2204
Epoch 3/230
141/141 [==============================] - 17s 119ms/step - loss: 1.3275 - accuracy: 0.5604 - val_loss: 2.2881 - val_accuracy: 0.2208
Epoch 4/230
141/141 [==============================] - 17s 120ms/step - loss: 1.3303 - accuracy: 0.5617 - val_loss: 2.2983 - val_accuracy: 0.2209
Epoch 5/230
141/141 [==============================] - 17s 118ms/step - loss: 1.3272 - accuracy: 0.5620 - val_loss: 2.2806 - val_accuracy: 0.2216
Epoch 6/230
141/141 [==============================] - 17s 117ms/step - loss: 1.3246 - accuracy: 0.5618 - val_loss: 2.2887 - val_accuracy: 0.2228
Epoch 7/230
141/141 [==============================] - 16s 117ms/step - loss: 1.3240 - accuracy: 0.5608 - val_loss: 2.3116 - val_accurac

In [ ]:
os.listdir('../../model')

In [ ]:
# bert_model.save('../../model/friends_model.h5')

In [ ]:
# model.save('cnn.h5')
# loaded_model = tf.keras.models.load_model('cnn.h5')

In [ ]:
type(bert_model)

In [37]:
# bert_model.save('../../model/friends_model.h5',save_format="tf")
bert_model.save_weights('../../model/friends_model_weights.h5')

In [ ]:
# loaded_model = tf.keras.models.load_model('cnn.h5')

test_load_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)
test_load_model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-9, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy()
metric = tf.keras.metrics.CategoricalCrossentropy('accuracy')

test_load_model.load_weights('../../model/friends_model_weights.h5')
test_load_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
bert_model.evaluate(input_test)

118/118 [==============================] - 3s 28ms/step - loss: 4.5699 - accuracy: 4.5699


[4.569864749908447, 4.569864749908447]

In [ ]:
test_load_model.evaluate(input_test)

118/118 [==============================] - 63s 535ms/step - loss: 4.5750 - accuracy: 4.5750


[4.574974536895752, 4.574974536895752]

## Testing

In [ ]:
def map_to_dict_testing(input_ids, attention_masks, token_type_ids):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
    }

def encode_test_dataset(ds, limit=-1):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds[limit-1:limit]
        print(ds)
    
    for text in ds.values:
        bert_input = text_to_feature(text)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
    
    print(len(ds.values))
        
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list)).map(map_to_dict_testing)

In [ ]:
data_test_sentence = encode_test_dataset(data_test['text'],5)

14    all of a sudden, the phone starts to ring. now...
Name: text, dtype: object
1


In [ ]:
data_test.head()

,text,label
1,"c'mon, you're going out with the guy! there's ...",5
3,"wait, does he eat chalk?",6
9,"then i look down, and i realize there's a phon...",4
11,that's right.,4
14,"all of a sudden, the phone starts to ring. now...",4


In [ ]:
data_test_sentence

<MapDataset shapes: {input_ids: (25,), token_type_ids: (25,), attention_mask: (25,)}, types: {input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}>

In [ ]:
XX = test_load_model.predict(data_test_sentence)

In [ ]:
XX[0][-1]

array([-0.42746884, -0.1117845 ,  0.1798155 , -0.21867178,  0.31221217,
        0.2932364 , -0.16892028], dtype=float32)